Chuẩn bị các thư viện

In [ ]:

!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.6/274.6 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 36.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 60.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a pro

Tải dữ liệu VizWiz

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nqa112/vizwiz-2023-edition")

print("Path to dataset files:", path)

100%|██████████| 17.5G/17.5G [03:40<00:00, 85.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1


Kiểm tra cấu trúc file của dữ liệu

In [ ]:
!ls /root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1

Annotations  test  train  val


Chia dữ liệu lại vì tập test không có kết quả, nên dùng valid thay cho test và một phần train để valid

In [ ]:
import os
import json
import shutil

# Định nghĩa đường dẫn
train_dir = "/root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1/train/train"
valid_dir = "/root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1/training_valid"
annotations_file = "/root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1/Annotations/train.json"
valid_annotations_file = "/root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1/Annotations/training_valid.json"

# Tạo thư mục valid nếu chưa có
os.makedirs(valid_dir, exist_ok=True)

# Đọc annotations từ file JSON
with open(annotations_file, "r") as f:
    annotations = json.load(f)

# Lấy danh sách tất cả file ảnh trong train (sắp xếp để giữ thứ tự)
train_files = sorted(os.listdir(train_dir))

# Tính số lượng file cần tách (10% cuối)
num_valid_samples = max(1, int(0.1 * len(train_files)))

# Lấy danh sách file ảnh cho valid (10% cuối)
valid_files = train_files[-num_valid_samples:]

# Di chuyển ảnh từ train sang valid
for file_name in valid_files:
    src_path = os.path.join(train_dir, file_name)
    dest_path = os.path.join(valid_dir, file_name)
    shutil.move(src_path, dest_path)

# Lọc annotations cho các file valid
valid_annotations = [ann for ann in annotations if ann["image"] in valid_files]

# Lưu annotations valid vào file JSON mới
with open(valid_annotations_file, "w") as f:
    json.dump(valid_annotations, f, indent=4)

# Cập nhật lại file annotations train (loại bỏ ảnh đã chuyển sang valid)
train_annotations = [ann for ann in annotations if ann["image"] not in valid_files]
with open(annotations_file, "w") as f:
    json.dump(train_annotations, f, indent=4)

print(f"Đã di chuyển {num_valid_samples} ảnh từ train sang valid và cập nhật annotations.")


Đã di chuyển 2395 ảnh từ train sang valid và cập nhật annotations.


Chỉnh lại đường dẫn dữ liệu cho dễ xài

In [ ]:
import os
import shutil

def move_and_cleanup(subset):
    src_dir = f"/root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1/{subset}/{subset}"
    # dst_dir = f"/root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1/{subset}"
    # src_dir = f"/root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1/{subset}"
    dst_dir = f"/root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1"

    # Kiểm tra nếu thư mục nguồn tồn tại
    if os.path.exists(src_dir) and os.path.isdir(src_dir):
        # Di chuyển tất cả ảnh lên thư mục cha
        for filename in os.listdir(src_dir):
            src_path = os.path.join(src_dir, filename)
            dst_path = os.path.join(dst_dir, filename)

            if os.path.isfile(src_path):  # Chỉ di chuyển file, tránh thư mục con
                shutil.move(src_path, dst_path)

        # Xóa thư mục `subset/subset` sau khi di chuyển xong
        os.rmdir(src_dir)

        print(f"✅ Đã di chuyển toàn bộ ảnh và xóa thư mục: {src_dir}")
    else:
        print(f"⚠️ Thư mục không tồn tại: {src_dir}")

# Thực hiện với các tập train, val, test
for subset in ["train", "val", "test"]:
    move_and_cleanup(subset)


✅ Đã di chuyển toàn bộ ảnh và xóa thư mục: /root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1/train/train
✅ Đã di chuyển toàn bộ ảnh và xóa thư mục: /root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1/val/val
✅ Đã di chuyển toàn bộ ảnh và xóa thư mục: /root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1/test/test


Xoá thư mục test

In [ ]:
import shutil
shutil.rmtree("/root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1/test")  # Xóa toàn bộ thư mục và nội dung bên trong

In [ ]:
import os

# Đường dẫn đến thư mục chứa ảnh (cập nhật theo dataset của bạn)
image_dir = "/root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1"

# Biến đếm số lượng ảnh đã xóa
deleted_count = 0

# Duyệt qua tất cả file trong thư mục
for file_name in os.listdir(image_dir):
    file_path = os.path.join(image_dir, file_name)

    # Kiểm tra nếu file là ảnh và có chứa "test" trong tên
    if "test" in file_name and file_name.lower().endswith((".jpg", ".jpeg", ".png")):
        os.remove(file_path)  # Xóa file
        deleted_count += 1

print(f"Tổng số ảnh đã xóa: {deleted_count}")


Tổng số ảnh đã xóa: 8000


Kiểm tra lại coi có còn tập test không (dùng Ctrl + F)

In [ ]:
!ls /root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1/

Streaming output truncated to the last 5000 lines.
VizWiz_train_00004767.jpg  VizWiz_train_00014538.jpg  VizWiz_val_00002750.jpg
VizWiz_train_00004768.jpg  VizWiz_train_00014539.jpg  VizWiz_val_00002751.jpg
VizWiz_train_00004769.jpg  VizWiz_train_00014540.jpg  VizWiz_val_00002752.jpg
VizWiz_train_00004770.jpg  VizWiz_train_00014541.jpg  VizWiz_val_00002753.jpg
VizWiz_train_00004771.jpg  VizWiz_train_00014542.jpg  VizWiz_val_00002754.jpg
VizWiz_train_00004772.jpg  VizWiz_train_00014543.jpg  VizWiz_val_00002755.jpg
VizWiz_train_00004773.jpg  VizWiz_train_00014544.jpg  VizWiz_val_00002756.jpg
VizWiz_train_00004774.jpg  VizWiz_train_00014545.jpg  VizWiz_val_00002757.jpg
VizWiz_train_00004775.jpg  VizWiz_train_00014546.jpg  VizWiz_val_00002758.jpg
VizWiz_train_00004776.jpg  VizWiz_train_00014547.jpg  VizWiz_val_00002759.jpg
VizWiz_train_00004777.jpg  VizWiz_train_00014548.jpg  VizWiz_val_00002760.jpg
VizWiz_train_00004778.jpg  VizWiz_train_00014549.jpg  VizWiz_val_00002761.jpg
VizWiz_train_

Mã hoá đặc trưng

In [ ]:
!touch /content/VizWiz-VQA-PyTorch/datasets/__init__.py

In [ ]:
!mkdir -p /content/VizWiz-VQA-PyTorch/prepro_data/

In [ ]:
!PYTHONPATH=/content/VizWiz-VQA-PyTorch python3 /content/VizWiz-VQA-PyTorch/preprocessing/image_features_extraction.py

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100% 230M/230M [00:01<00:00, 180MB/s]
Found 29309 images in /root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will 

In [ ]:
!mkdir -p /content/VizWiz-VQA-PyTorch/prepro_data

In [ ]:
!touch /content/VizWiz-VQA-PyTorch/prepro_data/vocabs.json

In [ ]:
!PYTHONPATH=/content/VizWiz-VQA-PyTorch python3 /content/VizWiz-VQA-PyTorch/preprocessing/create_vocabs.py

{'annotations': {'dir': '/root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1/Annotations',
                 'max_length': 26,
                 'min_count_word': 0,
                 'path_vocabs': './prepro_data/vocabs.json',
                 'top_ans': 3000},
 'images': {'arch': 'ResNet152',
            'dir': '/root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1',
            'img_size': 448,
            'mode': 'att',
            'path_features': './prepro_data/resnet14x14.h5',
            'preprocess_batch_size': 4,
            'preprocess_data_workers': 4},
 'logs': {'dir_logs': 'logs/vizwiz/'},
 'model': {'attention': {'dropout': 0.5, 'glimpses': 2, 'mid_features': 512},
           'classifier': {'dropout': 0.5},
           'pooling': {'dim_h': 1024,
                       'dim_q': 1024,
                       'dim_v': 2048,
                       'dropout_q': 0.5,
                       'dropout_v': 0.5},
           'pretrained_model': None,
   

Huấn luyện mô hình

In [ ]:
!PYTHONPATH=/content/VizWiz-VQA-PyTorch python3 /content/VizWiz-VQA-PyTorch/train.py

Model logs will be saved in logs/vizwiz/2025-03-17_11:10:42
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
train E000:   0% 0/152 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnin

Dự đoán kết quả trên tập Valid

In [ ]:
import json
import os

# Đảm bảo thư mục tồn tại
os.makedirs("./predictions", exist_ok=True)

# Đường dẫn file
file_path = "./predictions/default_pred.json"

# Dữ liệu mẫu
data = {"predictions": []}

# Ghi dữ liệu vào file JSON
with open(file_path, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

print(f"File {file_path} đã được tạo thành công!")

File ./predictions/default_pred.json đã được tạo thành công!


In [ ]:
!python predict.py

Loading samples to predict from /root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1/Annotations/val.json
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Loading Model from /content/final_log.pth
  0% 0/34 [00:00<?, ?it/s]Evaluating...

100% 34/34 [00:32<00:00,  1.04it/s]
Evaluation completed
Submission file saved in ./predictions/default_pred.json


In [ ]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

import json

split = 'val'
results = json.load(open('./predictions/default_pred.json'))
dataset = json.load(open('/root/.cache/kagglehub/datasets/nqa112/vizwiz-2023-edition/versions/1/Annotations/%s.json'%split))
img2gt = {x['image']:x['answers'] for x in dataset}

In [ ]:
print(results[10])
print(dataset[0])

{'image': 'VizWiz_val_00000010.jpg', 'answer': 'unanswerable'}
{'image': 'VizWiz_val_00000000.jpg', 'question': 'Ok. There is another picture I hope it is a better one.', 'answers': [{'answer': 'unanswerable', 'answer_confidence': 'yes'}, {'answer': 'unanswerable', 'answer_confidence': 'yes'}, {'answer': 'unanswerable', 'answer_confidence': 'yes'}, {'answer': 'unanswerable', 'answer_confidence': 'yes'}, {'answer': 'unanswerable', 'answer_confidence': 'maybe'}, {'answer': 'unanswerable', 'answer_confidence': 'yes'}, {'answer': 'unanswerable', 'answer_confidence': 'yes'}, {'answer': 'unanswerable', 'answer_confidence': 'no'}, {'answer': 'cannot repair this computer automatically', 'answer_confidence': 'maybe'}, {'answer': 'blank screen', 'answer_confidence': 'yes'}], 'answer_type': 'unanswerable', 'answerable': 0}


In [ ]:
from collections import Counter
img2ans_type = {}
for one_data in dataset:
    ans_counter = Counter([x['answer'] for x in one_data['answers']])
    ans = ans_counter.most_common(1)[0][0]
    if ans == 'yes' or ans == 'no':
        ans_type = 'yes/no'
    elif ans == 'unanswerable' or ans == 'unsuitable':
        ans_type = 'unanswerable'
    elif ans.isdigit():
        ans_type = 'number'
    else:
        ans_type = 'other'
    img2ans_type[one_data['image']] = ans_type

# all_ans = img2ans_type.values()
all_ans = list(img2ans_type.values())

print(len(all_ans))
for ans_type in set(all_ans):
    print(ans_type, ':', all_ans.count(ans_type)*1.0/len(all_ans))

4319
yes/no : 0.0476962259782357
unanswerable : 0.4332021301227136
other : 0.5091456355637879
number : 0.009956008335262792


In [ ]:
import numpy as np
img2acc = {}
imdir='VizWiz_%s_%012d.jpg'
import json

for pred in results:
    #img = imdir%(split,pred['question_id']) #Reconstruct image name from question_id
    img = pred['image']
    # print(img)
    # if img not in img2gt or not img2gt[img]:  # Không có GT, bỏ qua
    #   continue
    pred_ans = pred['answer']
    # print(pred_ans)
    # print("+")
    gt_ans = img2gt[img]
    gt_ans = [x['answer'] for x in gt_ans]
    gt_ans = [x.lower() for x in gt_ans]
    cur_acc = np.minimum(1.0, gt_ans.count(pred_ans)/3.0)
    img2acc[img] = cur_acc

accuracy = np.mean(list(img2acc.values()))  # Chuyển dict_values thành list trước khi tính trung bình
print("Accuracy:", accuracy)

# for ans_type in set(all_ans):
#     acc_per_type = np.mean([acc for img, acc in img2acc.items() if img2ans_type[img] == ans_type])
#     print(ans_type, ':', acc_per_type)

acc_values = [acc for img, acc in img2acc.items() if img2ans_type.get(img) == ans_type]

if acc_values:  # Kiểm tra danh sách có phần tử trước khi tính trung bình
    acc_per_type = np.mean(acc_values)
else:
    acc_per_type = 0.0  # Hoặc np.nan nếu bạn muốn bỏ qua

print(ans_type, ":", acc_per_type)

Accuracy: 0.6131820637493246
number : 0.25581395348837216
